In [1]:
import torch
import torch.nn.functional as F
import sympy as sp
import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv('physics_equations.csv')

functions = []
num_vars_per_func = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# variables']
    function_details = {
        'formula': formula,
        'variables': []
    }
    
    for i in range(1, 11):  
        v_name = row.get(f'v{i}_name', None)
        v_low = row.get(f'v{i}_low', None)
        v_high = row.get(f'v{i}_high', None)
        
        if pd.notna(v_name):
            function_details['variables'].append({
                'name': v_name,
                'low': v_low,
                'high': v_high
            })
    
    functions.append(function_details)
    num_vars_per_func.append(num_vars)

'''for i, func in enumerate(functions):
    print(f"Function {i+1}:")
    print(f"  Formula: {func['formula']}")
    print(f"  Number of Variables: {num_vars_per_func[i]}")
    print("  Variables:")
    for var in func['variables']:
        print(f"    - Name: {var['name']}, Range: ({var['low']}, {var['high']})")
    print()'''

'for i, func in enumerate(functions):\n    print(f"Function {i+1}:")\n    print(f"  Formula: {func[\'formula\']}")\n    print(f"  Number of Variables: {num_vars_per_func[i]}")\n    print("  Variables:")\n    for var in func[\'variables\']:\n        print(f"    - Name: {var[\'name\']}, Range: ({var[\'low\']}, {var[\'high\']})")\n    print()'

In [4]:
def generate_function(function, sample_size, x, max_vars, device):
    sympy_symbols = []
    param_tensors = []
    
    for var in function["variables"]:
        sym = sp.symbols(var["name"])
        sympy_symbols.append(sym)
        min_val, max_val = var["low"], var["high"]
        param = (max_val - min_val) * torch.rand(sample_size, 1, device=device) + min_val
        param_tensors.append(param)

    sympy_symbols.append(sp.symbols('x'))
    params = torch.cat(param_tensors, dim=1)
    padded_params = F.pad(params, pad=(0, max_vars - params.size(1)))
    padded_params = padded_params.expand(sample_size, max_vars)
    formula = sp.sympify(function["formula"])
    eval_func = sp.lambdify(sympy_symbols, formula, modules="numpy")
        
    results = []
    for xi in x:
        input_values = torch.cat([params, xi.expand(sample_size, 1)], dim=1)
        results.append(eval_func(*input_values.T))
    results = torch.stack(results, dim=1)
    
    return results, formula, sympy_symbols, padded_params

In [5]:
sample_size = 1000
sequence_length = 100
num_funcs = 10
max_vars = 5

x_values = torch.linspace(-1, 1, sequence_length).to(device)
hold = []
for f in functions[0:10]:
    try:
        results = generate_function(f, sample_size, x_values, max_vars, device)
        hold.append(results)
    except Exception as e:
        print(f"Error processing function {f}: {e}")

In [6]:
y_values = torch.stack([l[0] for l in hold])
formulas = [l[1] for l in hold]
symbols = [l[2] for l in hold]
param_values = torch.stack([l[3] for l in hold])
num_params = torch.tensor([len(l[2]) for l in hold])

In [23]:
def evaluate_function(params, symbols, formula, x):
    var_values = {symbols[j]: params[:, j] for j in range(len(symbols)-1)}
    for key in var_values:
        print(f"{key}: {var_values[key][0]}")
    eval_func = sp.lambdify(symbols, formula, modules="numpy")
    results = []
    for xi in x:
        var_values[symbols[-1]] = xi
        np_values = {str(sym): var_values[sym].detach().cpu().numpy() for sym in symbols}
        results.append(eval_func(**np_values))
    tensor_results = [torch.tensor(r, device=device) for r in results]
    return torch.stack(tensor_results, dim=1)

In [10]:
epsilon = 1e-6
derivatives = torch.zeros(num_funcs, sample_size, sequence_length, max_vars)
for f in range(num_funcs):
    params_f = param_values[f].clone().detach().requires_grad_(True)
    for p in range(len(symbols[f])):
        perturbed_params_pos = params_f.clone()
        perturbed_params_neg = params_f.clone()
        perturbed_params_pos[:,p] += epsilon
        forward_values = evaluate_function(perturbed_params_pos, symbols[f], formulas[f], x_values)
        perturbed_params_neg[:, p] -= 2*epsilon
        backward_values = evaluate_function(perturbed_params_neg, symbols[f], formulas[f], x_values)
        derivatives[f, :, :, p] = (forward_values - backward_values) / (2 * epsilon)
derivatives.shape


params recieved: 0.2693064510822296
a: 0.2693064510822296
b: 0.8719912767410278
c: 0.4585850238800049

params recieved: 0.26930344104766846
a: 0.26930344104766846
b: 0.8719912767410278
c: 0.4585850238800049

params recieved: 0.26930543780326843
a: 0.26930543780326843
b: 0.871992290019989
c: 0.4585850238800049

params recieved: 0.26930543780326843
a: 0.26930543780326843
b: 0.8719892501831055
c: 0.4585850238800049

params recieved: 0.26930543780326843
a: 0.26930543780326843
b: 0.8719912767410278
c: 0.45858603715896606

params recieved: 0.26930543780326843
a: 0.26930543780326843
b: 0.8719912767410278
c: 0.4585830271244049

params recieved: 0.26930543780326843
a: 0.26930543780326843
b: 0.8719912767410278
c: 0.4585850238800049

params recieved: 0.26930543780326843
a: 0.26930543780326843
b: 0.8719912767410278
c: 0.4585850238800049

params recieved: -0.7356988191604614
k: -0.7356988191604614

params recieved: -0.735701858997345
k: -0.735701858997345

params recieved: -0.7356998324394226
k: -

torch.Size([10, 1000, 100, 5])

In [16]:
hessians = torch.zeros(num_funcs, sample_size, sequence_length, max_vars, max_vars)
for f in range(num_funcs):
    params_f = param_values[f].clone().detach().requires_grad_(True)
    for j in range(len(symbols[f])):
        perturbed_pp = params_f.clone()
        perturbed_pn = params_f.clone()
        perturbed_np = params_f.clone()
        perturbed_nn = params_f.clone()

        perturbed_pp[:, j] += epsilon
        perturbed_pn[:, j] += epsilon
        perturbed_np[:, j] -= epsilon
        perturbed_nn[:, j] -= epsilon
        for k in range(len(symbols[f])):
            perturbed_pp[:, k] += epsilon
            perturbed_pn[:, k] -= epsilon
            perturbed_np[:, k] += epsilon
            perturbed_nn[:, k] -= epsilon

            forward_forward = evaluate_function(perturbed_pp, symbols[f], formulas[f], x_values)
            forward_backward = evaluate_function(perturbed_pn, symbols[f], formulas[f], x_values)
            backward_forward = evaluate_function(perturbed_np, symbols[f], formulas[f], x_values)
            backward_backward = evaluate_function(perturbed_nn, symbols[f], formulas[f], x_values)
            hessians[f, :, :, j, k] = (forward_forward - forward_backward - backward_forward + backward_backward) / (4 * epsilon **2)
hessians.shape

torch.Size([10, 1000, 100, 5, 5])

In [29]:
#Problem with params revieced!
epsilon = 1e-4
f = 5
tester = torch.zeros(num_funcs, sample_size, sequence_length, max_vars)
parami = param_values[f].clone().detach().requires_grad_(True)
for p in range(len(symbols[f])):
    plus = parami.clone()
    minus = parami.clone()

    print(f"\nInitial {p}: {plus[0, p]}")
    plus[:,p] += epsilon
    print(f"forward {p}: {plus[0, p]}")
    forward_values = evaluate_function(plus, symbols[f], formulas[f], x_values)
    print(f"eval forward {p}: {forward_values[0, 0]}")
    minus[:, p] -= epsilon
    print(f"\nbackward {p}: {minus[0, p]}")
    backward_values = evaluate_function(minus, symbols[f], formulas[f], x_values)
    print(f"eval backward {p}: {backward_values[0, 0]}")
    tester[f, :, :, p] = (forward_values - backward_values) / (2 * epsilon)
    print(f"\nDerivative {p}: {tester[f, 0, 0, p]}")
tester.shape


Initial 0: 0.3012508451938629
forward 0: 0.30135083198547363
m: 0.30135083198547363
eval forward 0: 0.15067541599273682

backward 0: 0.3011508584022522
m: 0.3011508584022522
eval backward 0: 0.1505754292011261

Derivative 0: 0.49993395805358887

Initial 1: 0.0
forward 1: 9.999999747378752e-05
m: 0.3012508451938629
eval forward 1: 0.15062542259693146

backward 1: -9.999999747378752e-05
m: 0.3012508451938629
eval backward 1: 0.15062542259693146

Derivative 1: 0.0


torch.Size([10, 1000, 100, 5])

In [30]:
tester[f, 0, 0]

tensor([0.4999, 0.0000, 0.0000, 0.0000, 0.0000])

In [31]:
x_values[f], formulas[f], symbols[f], param_values[f, 0], y_values[f, 0, 0]


(tensor(-0.8990, device='cuda:4'),
 m*x**2/2,
 [m, x],
 tensor([0.3013, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:4'),
 tensor(0.1506, device='cuda:4'))

In [182]:
def compute_derivatives(parameters, func, symbols, formula, x, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = len(symbols)
    gradients = torch.zeros(batch_size, num_params, x.shape[0])
    print(parameters.shape)
    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        print(param_tensor.shape)
        for j in range(num_params):
            perturbed_params_pos = param_tensor.clone()
            perturbed_params_neg = param_tensor.clone()
            print(perturbed_params_pos.shape)
            perturbed_params_pos[j] += epsilon
            forward_values = func(perturbed_params_pos, symbols, formula, x)
            print(forward_values.shape)
            perturbed_params_neg[j] -= epsilon
            backward_values = func(perturbed_params_neg, symbols, formula, x)
            
            gradients[i, j] = (forward_values - backward_values) / (2 * epsilon)
    
    return gradients


In [72]:
def compute_hessians(parameters, func, symbols, formula, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = parameters.shape[2]
    hessian = torch.zeros(batch_size, num_params, num_params)

    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(num_params):
            for k in range(j, num_params):
                perturbed_params_pp = param_tensor.clone()
                perturbed_params_pp[0, j, 0] += epsilon
                perturbed_params_pp[0, k, 0] += epsilon
                ff_value = func(perturbed_params_pp, symbols, formula)

                perturbed_params_nn = param_tensor.clone()
                perturbed_params_nn[0, j, 0] -= epsilon
                perturbed_params_nn[0, k, 0] -= epsilon
                bb_value = func(perturbed_params_nn, symbols, formula)

                if j == k:
                    center_value = func(param_tensor, symbols, formula)
                    hessian[i, j, k] = (ff_value - 2 * center_value + bb_value) / (epsilon ** 2)
                else:
                    perturbed_params_pn = param_tensor.clone()
                    perturbed_params_pn[0, j, 0] += epsilon
                    perturbed_params_pn[0, k, 0] -= epsilon
                    fb_value = func(perturbed_params_pn, symbols, formula)

                    perturbed_params_np = param_tensor.clone()
                    perturbed_params_np[0, j, 0] -= epsilon
                    perturbed_params_np[0, k, 0] += epsilon
                    bf_value = func(perturbed_params_np, symbols, formula)

                    hessian[i, j, k] = (ff_value - fb_value - bf_value + bb_value) / (4 * epsilon ** 2)
                    hessian[i, k, j] = hessian[i, j, k]

    return hessian

In [118]:
param_values[0].shape, symbols_hold[0], formulas[0]

(torch.Size([1000, 5]), [a, b, c, x], a*x**2 + b*x + c)

In [8]:
'''def compute_hessians(parameters, func, symbols, formula, epsilon=1e-6): #need to fix
    batch_size = parameters.shape[0]
    max_num_params = parameters.shape[2]
    hessians = torch.zeros((batch_size, max_num_params, max_num_params))
    
    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(max_num_params):
            for k in range(max_num_params):
                # Perturb j-th and k-th parameters
                perturbed_params = param_tensor.clone()
                
                # Compute f(x + epsilon * e_j + epsilon * e_k)
                perturbed_params_jk = perturbed_params.clone()
                perturbed_params_jk[:, j, :] += epsilon
                perturbed_params_jk[:, k, :] += epsilon
                f_plus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x + epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_plus_minus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j + epsilon * e_k)
                perturbed_params_jk[:, j, :] -= 2 * epsilon
                perturbed_params_jk[:, k, :] += 2 * epsilon
                f_minus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_minus_minus = func(perturbed_params_jk, symbols, formula)
                hessians[i, j, k] = (f_plus_plus - f_plus_minus - f_minus_plus + f_minus_minus) / (4 * epsilon**2)
    
    return hessians'''

In [42]:
flat_result = torch.flatten(torch.stack(y_values).unsqueeze(2), start_dim=0, end_dim=1)

max_der = max(d.size(1) for d in derivatives)
padded_ders = []
for d in derivatives:
    padded_tensor = F.pad(d, pad=(0, max_der - d.size(1)))
    padded_ders.append(padded_tensor)
flat_der = torch.flatten(torch.stack(padded_ders), start_dim=0, end_dim=1)

max_hess = max_der**2
padded_hess = []
for h in hessians:
    h = torch.flatten(h, start_dim=1)
    padded_tensor = F.pad(h, pad=(0, max_hess - h.size(1)))
    padded_hess.append(padded_tensor)
flat_hess = torch.flatten(torch.stack(padded_hess), start_dim=0, end_dim=1)

flat_der = F.pad(flat_der, (0, flat_hess.size(-1) - flat_der.size(-1)))
flat_result = F.pad(flat_result, (0, flat_hess.size(-1) - flat_result.size(-1)))

flat_result.shape, flat_der.shape, flat_hess.shape

(torch.Size([10000, 81]), torch.Size([10000, 81]), torch.Size([10000, 81]))

In [ ]:

num_params = torch.repeat_interleave(num_params, repeats=sample_size)
y_values = torch.flatten(y_values, end_dim=1)
param_values = torch.flatten(param_values, end_dim=1)
formulas = np.array([[formula for _ in range(1000)] for formula in formulas]).flatten()
symbols = []
for s in symbols_hold:
    for _ in range(sample_size):
        symbols.append(s)

y_values.shape, param_values.shape, num_params.shape, formulas.shape, len(symbols)

In [44]:
torch.save({
    'flattened_data': torch.stack([flat_result, flat_der, flat_hess], dim=2),
    'results': y_values,
    'formulas': hold_formulas,
    'symbols': hold_symbols,
    'params': param_values,
    'num_params': num_params,
}, 'hold_data.pth')